# Fine-tuning LLMs

Many LLMs are general purpose models trained on a broad range of data and use cases. This enables them to perform well in a variety of applications, as shown in previous modules. It is not uncommon though to find situations where applying a general purpose model performs unacceptably for specific dataset or use case. This often does not mean that the general purpose model is unusable. Perhaps, with some new data and additional training the model could be improved, or fine-tuned, such that it produces acceptable results for the specific use case.

Fine-tuning uses a pre-trained model as a base and continues to train it with a new, task targeted dataset. Conceptually, fine-tuning leverages that which has already been learned by a model and aims to focus its learnings further for a specific task.

It is important to recognize that fine-tuning is model training. The training process remains a resource intensive, and time consuming effort. Albeit fine-tuning training time is greatly shortened as a result of having started from a pre-trained model. The model training process can be accelerated through the use of tools like Microsoft's [DeepSpeed](https://github.com/microsoft/DeepSpeed).

This notebook will explore how to perform fine-tuning at scale.

Learning Objectives
1. Prepare a novel dataset
1. Fine-tune the `t5-small` model to classify movie reviews.
1. Leverage DeepSpeed to enhance training process.

In [1]:
!/usr/local/cuda/bin/nvcc --version

!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Sun Oct  1 03:32:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |       

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-4116e9cf-ad78-f52f-cd0c-552480b31443)


Creating a local temporary directory on the Driver. This will serve as a root directory for the intermediate model checkpoints created during the training process. The final model will be persisted to DBFS.


In [7]:
import tempfile

tmpdir = tempfile.TemporaryDirectory()
local_training_root = tmpdir.name

In [ ]:
!pip install transformers datasets

In [2]:
import os
import pandas as pd
import transformers as tr
from datasets import load_dataset

### Step 1 - Data Preparation

The first step of the fine-tuning process is to identify a specific task and supporting dataset. In this notebook, we will consider the specific task to be classifying movie reviews. This idea is generally simple task where a movie review is provided as plain-text and we would like to determine whether or not the review was positive or negative.

The [IMDB dataset](https://huggingface.co/datasets/imdb) can be leveraged as a supporting dataset for this task. The dataset conveniently provides both a training and testing dataset with labeled binary sentiments, as well as a dataset of unlabeled data.



In [12]:
imdb_ds = load_dataset("imdb")

### Step 2 - Select pre-trained model

The next step of the fine-tuning process is to select a pre-trained model. We will consider using the [T5](https://huggingface.co/docs/transformers/model_doc/t5) [[paper]](https://arxiv.org/pdf/1910.10683.pdf) family of models for our fine-tuning purposes. The T5 models are text-to-text transformers that have been trained on a multi-task mixture of unsupervised and supervised tasks. They are well suited for tasks such as summarization, translation, text classification, question answering, and more.

The `t5-small` version of the T5 models has 60 million parameters. This slimmed down version will be sufficient for our purposes.


In [13]:
model_checkpoint = "t5-small"
tokenizer = tr.AutoTokenizer.from_pretrained(model_checkpoint)


As mentioned above, the IMDB dataset is a binary sentiment dataset. Its labels therefore are encoded as (-1 - unknown; 0 - negative; 1 - positive) values. In order to use this dataset with a text-to-text model like T5, the label set needs to be represented as a string. There are a number of ways to accomplish this. Here, we will simply translate each label id to its corresponding string value.


In [14]:
def to_tokens(
    tokenizer: tr.models.t5.tokenization_t5_fast.T5TokenizerFast, label_map: dict
) -> callable:
    """
    Given a `tokenizer` this closure will iterate through `x` and return the result of `apply()`.
    This function is mapped to a dataset and returned with ids and attention mask.
    """

    def apply(x) -> tr.tokenization_utils_base.BatchEncoding:
        """From a formatted dataset `x` a batch encoding `token_res` is created."""
        target_labels = [label_map[y] for y in x["label"]]
        token_res = tokenizer(
            x["text"],
            text_target=target_labels,
            return_tensors="pt",
            truncation=True,
            padding=True,
        )
        return token_res

    return apply

In [16]:
imdb_label_lookup = {0: "negative", 1: "positive", -1: "unknown"}

imdb_to_tokens = to_tokens(tokenizer, imdb_label_lookup)
tokenized_dataset = imdb_ds.map(
    imdb_to_tokens, batched=True, remove_columns=["text", "label"]
)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset["train"]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 25000
})

### Step 3 - Setup Training

The model training process is highly configurable. The [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) class effectively exposes the configurable aspects of the process allowing one to customize them accordingly. Here, we will focus on setting up a training process that performs a single epoch of training with a batch size of 16. We will also leverage `adamw_torch` as the optimizer.


In [ ]:
!pip install accelerate==0.20.3
#!pip install transformers[torch]

In [9]:
checkpoint_name = "test-trainer"
local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)
training_args = tr.TrainingArguments(
    local_checkpoint_path,
    num_train_epochs=1,  # default number of epochs to train is 3
    per_device_train_batch_size=16,
    optim="adamw_torch",
    report_to=["tensorboard"],
)

In [10]:
model = tr.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, cache_dir="sample_data/"
)

In [11]:
data_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)
trainer = tr.Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

### Step 4 - Train

Before starting the training process, let's turn on Tensorboard. This will allow us to monitor the training process as checkpoint logs are created.


In [12]:
tensorboard_display_dir = f"{local_checkpoint_path}/runs"

In [13]:
trainer.train()

# save model to the local checkpoint
trainer.save_model()
trainer.save_state()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.611000
1000,0.137700
1500,0.126900


In [14]:
# persist the fine-tuned model to DBFS
final_model_path = f"/sample_data/llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)

### Step 5 - Predict

In [ ]:
fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)

In [16]:
reviews = [
    """
'Despicable Me' is a cute and funny movie, but the plot is predictable and the characters are not very well-developed. Overall, it's a good movie for kids, but adults might find it a bit boring.""",
    """ 'The Batman' is a dark and gritty take on the Caped Crusader, starring Robert Pattinson as Bruce Wayne. The film is a well-made crime thriller with strong performances and visuals, but it may be too slow-paced and violent for some viewers.
""",
    """
The Phantom Menace is a visually stunning film with some great action sequences, but the plot is slow-paced and the dialogue is often wooden. It is a mixed bag that will appeal to some fans of the Star Wars franchise, but may disappoint others.
""",
    """
I'm not sure if The Matrix and the two sequels were meant to have a tigh consistency but I don't think they quite fit together. They seem to have a reasonably solid arc but the features from the first aren't in the second and third as much, instead the second and third focus more on CGI battles and more visuals. I like them but for different reasons, so if I'm supposed to rate the trilogy I'm not sure what to say.
""",
]
inputs = tokenizer(reviews, return_tensors="pt", truncation=True, padding=True)
pred = fine_tuned_model.generate(
    input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
)

# COMMAND ----------

pdf = pd.DataFrame(
    zip(reviews, tokenizer.batch_decode(pred, skip_special_tokens=True)),
    columns=["review", "classification"],
)
display(pdf)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


,review,classification
0,"\n'Despicable Me' is a cute and funny movie, b...",positive
1,'The Batman' is a dark and gritty take on the...,positive
2,\nThe Phantom Menace is a visually stunning fi...,positive
3,\nI'm not sure if The Matrix and the two seque...,negative


## DeepSpeed

As model architectures evolve and grow, they continually push the limits of available computational resources. For example, some large LLMs having hundreds of billions of parameters making them too large to fit, in some cases, in available GPU memory. Models of this scale therefore need to leverage distributed processing or high-end hardware, and sometimes even both, to support training efforts. This makes large model training a costly undertaking, and therefore accelerating the training process is highly desirable.

As mentioned above, one such framework that can be leveraged to accelerate the model training process is Microsoft's [DeepSpeed](https://github.com/microsoft/DeepSpeed) [[paper]](https://arxiv.org/pdf/2207.00032.pdf). This framework provides advances in compression, distributed training, mixed precision, gradient accumulation, and checkpointing.

It is worth noting that DeepSpeed is intended for large models that do not fit into device memory. The `t5-base` model we are using is not a large model, and therefore DeepSpeed is not expected to provide a benefit.


### Environment Setup

The intended use for DeepSpeed is in a distributed compute environment. As such, each node of the environment is assigned a `rank` and `local_rank` in relation to the size of the distributed environment.

Here, since we are testing with a single node/GPU environment we will set the `world_size` to 1, and both `ranks` to 0.


In [4]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9995"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"


### Configuration

There are a number of [configuration options](https://www.deepspeed.ai/docs/config-json/) that can be set to enhance the training and inference processes. The [ZeRO optimization](https://www.deepspeed.ai/training/#memory-efficiency) settings target reducing the memory footprint allowing for larger models to be efficiently trained on limited resources.

The Hugging Face `TrainerArguments` accept the configuration either from a JSON file or a dictionary. Here, we will define the dictionary.


In [43]:
zero_config = {
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {"device": "cpu", "pin_memory": True},
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto",
            "torch_adam": True,
        },
    },
    "scheduler": {
      "type": "WarmupLR",
      "params": {
          "warmup_min_lr": "auto",
          "warmup_max_lr": "auto",
          "warmup_num_steps": "auto"
      }
  },
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu":"auto"
}

In [19]:
model_checkpoint = "t5-base"
tokenizer = tr.AutoTokenizer.from_pretrained(
    model_checkpoint, cache_dir="sample_data/"
)


model = tr.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, cache_dir="sample_data/"
)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


There are only two changes made to the training setup from above. The first is to set a new checkpoint name. The second is to add the `deepspeed` configuration to the `TrainingArguments`.

Note: at this time the `deepspeed` argument is considered an experimental feature and may evolve in the future.

In [ ]:
!pip3 install deepspeed==0.9.3

In [ ]:
checkpoint_name = "test-trainer-deepspeed"
checkpoint_location = os.path.join(local_training_root, checkpoint_name)
training_args = tr.TrainingArguments(
    checkpoint_location,
    num_train_epochs=3,  # default number of epochs to train is 3
    per_device_train_batch_size=8,
    deepspeed=zero_config,  # add the deepspeed configuration
    report_to=["tensorboard"],
)

data_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)
trainer = tr.Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

trainer.save_model()
trainer.save_state()

Rank: 0 partition count [1] and sizes[(222903552, False)] 


In [ ]:
final_model_path = f"/sample_data/llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)

In [ ]:
review = [
    """
           I'm not sure if The Matrix and the two sequels were meant to have a tight consistency but I don't think they quite fit together. They seem to have a reasonably solid arc but the features from the first aren't in the second and third as much, instead the second and third focus more on CGI battles and more visuals. I like them but for different reasons, so if I'm supposed to rate the trilogy I'm not sure what to say."""
]
inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)

pred = fine_tuned_model.generate(
    input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
)

In [ ]:
pdf = pd.DataFrame(
    zip(review, tokenizer.batch_decode(pred, skip_special_tokens=True)),
    columns=["review", "classification"],
)
display(pdf)